In [5]:
import cv2
import numpy as np

widthImg = 480
heightImg = 640
count = 0

#For live capture of document {
# cap = cv2.VideoCapture(0)
# cap.set(3,640)
# cap.set(4,480)
# cap.set(10,150)
#}

def presProcessing(img):
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray,(5,5),1)
    imgCanny = cv2.Canny(imgBlur,100,100)
    kernel = np.ones((5,5))
    imgDilation = cv2.dilate(imgCanny,kernel,iterations=2)
    imgThreshhold = cv2.erode(imgDilation,kernel,iterations=1)

    # cv2.imshow("Gray",imgGray)

    return imgThreshhold

def getContours(img):
    biggest = np.array([])
    maxArea = 0
    contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>5000:
            cv2.drawContours(imgContour,cnt,-1,(255,0,0),3)
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True)
            if area>maxArea and len(approx) == 4:
                biggest = approx
                maxArea = area
   
    cv2.drawContours(imgBiggestContour,biggest,-1,(255,0,0),20)
    return biggest

def reorder(myPoints):
    myPoints = myPoints.reshape((4,2))
    myPointsNew = np.zeros((4,1,2),np.int32)
    add = myPoints.sum(1)
    # print("add",add)

    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]
    diff = np.diff(myPoints,axis=1)
    myPointsNew[1] = myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    # print(myPointsNew)

    return myPointsNew

def getWarp(img,biggest):
    # biggest = np.array([[[440,134]],[[307,145]],[[298,438]],[[439,440]]])
    biggest = reorder(biggest)
    # print(biggest.shape)
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0,0],[widthImg,0],[0,heightImg],[widthImg,heightImg]])
    matrix = cv2.getPerspectiveTransform(pts1,pts2)
    imgOutput = cv2.warpPerspective(img,matrix,(widthImg,heightImg))

    imgCropped = imgOutput[10:imgOutput.shape[0]-10,10:imgOutput.shape[1]-10]
    imgCropped = cv2.resize(imgCropped,(widthImg,heightImg))

    return imgCropped

while True:
    # success,img = cap.read()
    img = cv2.imread("images/document.jpg")
    img = cv2.resize(img,(widthImg,heightImg))
    imgContour = img.copy()
    imgBiggestContour = img.copy()

    imgThres = presProcessing(img)
    biggest = getContours(imgThres)
    # print(biggest)

    # cv2.imshow("Video",img)
    cv2.imshow("Contour",imgContour)
    cv2.imshow("Corner Points",imgBiggestContour)

    imgWarped = img.copy()

    if(biggest.size!=0):
        imgWarped = getWarp(img,biggest)
        # cv2.imshow("Warped Image",imgWarped)
        imgWarpedGray = cv2.cvtColor(imgWarped,cv2.COLOR_BGR2GRAY)
        imgWarpedBlur = cv2.GaussianBlur(imgWarpedGray,(5,5),1)
        thresh = cv2.adaptiveThreshold(imgWarpedBlur, 255,
	    cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)
        # cv2.imshow("Mean Adaptive Thresholding", thresh)

    key = cv2.waitKey(1) & 0xFF
    if key==ord('s'):
        filename = "scanneddoc" + str(count) + ".jpg"
        cv2.imwrite(filename,imgWarped)
        count+=1
    if key==ord('q'):
        break


# cap.release()
cv2.destroyAllWindows()